In [1]:
import json
import os
import random
import time

In [2]:
import h3
import geopandas as gpd
import pandas as pd

In [3]:
from kroger_api import KrogerAPI
from kroger_api.utils.env import load_and_validate_env, get_zip_code
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [4]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [5]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [6]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [7]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [8]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Store Data

In [9]:
load_and_validate_env(["KROGER_CLIENT_ID", "KROGER_CLIENT_SECRET"])
zip_code = get_zip_code(default="10001")

In [10]:
kroger = KrogerAPI()
token_info = kroger.authorization.get_token_with_client_credentials("product.compact")

Found saved token, will test if it's still valid...
Token appears invalid, requesting a new one


In [11]:
res = 3
distance = 0.5

usa_geom = states_df[~states_df["STUSPS"].isin(["PR", "VI", "GU", "MP", "AS"])][
    "geometry"
].union_all()

h3_shape = h3.geo_to_h3shape(usa_geom.buffer(distance=distance))
h3_cells = h3.h3shape_to_cells(h3_shape, res=res)
geojson = h3.cells_to_geo(h3_cells)

In [12]:
stores = []
for cell in tqdm(h3_cells, desc="Parsing Stores"):
    lat, lng = h3.cell_to_latlng(cell)
    locations = kroger.location.search_locations(
        lat=lat, lon=lng, radius_in_miles=100, limit=200
    )
    if len(locations["data"]):
        for location in locations["data"]:
            location["geometry"] = Point(
                location["geolocation"]["longitude"],
                location["geolocation"]["latitude"],
            )
            location["STUSPS"] = location["address"]["state"]
            stores.append(location)
    else:
        continue
    time.sleep(random.uniform(0.01, 0.5))

Parsing Stores:   0%|          | 0/911 [00:00<?, ?it/s]

In [45]:
stores_gdf = gpd.GeoDataFrame(stores, crs=4326)
stores_gdf = stores_gdf.drop_duplicates("locationId")
stores_gdf.to_file("data/all_stores.gpkg")

In [52]:
stores_gdf = gpd.read_file("data/all_stores.gpkg")

In [53]:
smiths_gdf = stores_gdf[stores_gdf["chain"] == "SMITHS"].reset_index(drop=True)

In [54]:
smiths_gdf = smiths_gdf.to_crs(32143)
smiths_gdf.to_file("data/stores.gpkg")

In [55]:
smiths_count_gdf = smiths_gdf.groupby("STUSPS").size().reset_index(name="COUNT")

## Combine With States

In [56]:
store_count_gdf = states_with_population_df.merge(
    smiths_count_gdf, on="STUSPS", how="left"
)

In [61]:
store_count_gdf = store_count_gdf.dropna()

In [62]:
store_count_gdf["per_100k"] = (
    store_count_gdf["COUNT"] / (store_count_gdf["POPULATION"] / 100_000)
).round(decimals=2)
store_count_gdf["per_1m"] = (
    store_count_gdf["COUNT"] / (store_count_gdf["POPULATION"] / 1_000_000)
).round(decimals=2)

In [63]:
store_count_gdf = store_count_gdf.to_crs(32143)
store_count_gdf.to_file(f"data/Smiths_Grocery_Per_State.gpkg")

In [64]:
store_count_gdf.sort_values("per_1m", ascending=False)

,STUSPS,NAME,POPULATION,geometry,COUNT,per_100k,per_1m
22,UT,Utah,3443222.0,"POLYGON ((274416.073 1920986.082, 274497.363 1...",56.0,1.63,16.26
28,NV,Nevada,3214363.0,"POLYGON ((-233304.553 2134292.88, -233127.26 2...",46.0,1.43,14.31
24,WY,Wyoming,585067.0,"POLYGON ((535264.098 2741376.081, 536015.388 2...",7.0,1.20,11.96
12,NM,New Mexico,2121164.0,"POLYGON ((735127.619 1239819.536, 735099.428 1...",24.0,1.13,11.31
31,MT,Montana,1131302.0,"POLYGON ((159953.554 3142118.013, 160030.284 3...",4.0,0.35,3.54
8,ID,Idaho,1971122.0,"POLYGON ((41359.954 2688609.954, 42007.529 268...",3.0,0.15,1.52
35,AZ,Arizona,7473027.0,"POLYGON ((186007.935 1357438.968, 186198.735 1...",4.0,0.05,0.54
